<a href="https://colab.research.google.com/github/SSenitha/CCS3052_Advance_DSA/blob/Dilini-s-Branch/Dijkstra_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Dataset

In [43]:
# Load data from the repository
!wget https://raw.githubusercontent.com/SSenitha/CCS3052_Advance_DSA/refs/heads/main/Cities_of_SriLanka.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
#Import necessary libraries

import math, time, heapq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

from collections import defaultdict
from sklearn.neighbors import BallTree

In [45]:
# Read the dataset and output the total count
#df = pd.read_csv('/content/Cities_of_SriLanka.csv')

#----------------For VS Code----------------
df = pd.read_csv('./Cities_of_SriLanka.csv')
#-------------------------------------------
print(f"Total rows count: {df['city id'].count()}")
df.rename(columns={"city id": "city_id"}, inplace=True)
df.head()

Total rows count: 2155


,city_id,district_id,name_en,name_si,name_ta,sub_name_en,sub_name_si,sub_name_ta,postcode,latitude,longitude
0,1,1,Akkaraipattu,අක්කරපත්තුව,அக்கரைப்பற்று,NULL,NULL,NULL,32400.0,7.218428,81.854116
1,2,1,Ambagahawatta,අඹගහවත්ත,அம்பகஹவத்த,NULL,NULL,NULL,90326.0,7.301756,81.674729
2,3,1,Ampara,අම්පාර,அம்பாறை,NULL,NULL,NULL,32000.0,7.301756,81.674729
3,4,1,Bakmitiyawa,බක්මිටියාව,பக்மிடியாவ,NULL,NULL,NULL,32024.0,7.029632,81.680205
4,5,1,Deegawapiya,දීඝවාපිය,தீகவாபி,NULL,NULL,NULL,32006.0,7.301756,81.674729


In [46]:
#Remove the Cities with little to no geographical difference

df = df.drop_duplicates(subset=['latitude', 'longitude'], keep='first').reset_index(drop=True)
print(f"Total rows count after dropping duplicates: {df['city_id'].count()}")

Total rows count after dropping duplicates: 1919


In [47]:
#Dropping the columns that won't be used in the making of graph/Matrix
df = df.drop(columns=["district_id","name_si","name_ta","sub_name_en","sub_name_si","sub_name_ta","postcode"])

In [48]:
def resetIndex():
  df.drop(columns=["city_id"], inplace=True)
  df.insert(0, "city_id", df.index)
  return df

In [49]:
resetIndex()
df.head()

,city_id,name_en,latitude,longitude
0,0,Akkaraipattu,7.218428,81.854116
1,1,Ambagahawatta,7.301756,81.674729
2,2,Bakmitiyawa,7.029632,81.680205
3,3,Digamadulla Weeragoda,7.390125,81.696588
4,4,Dorakumbura,7.358870,81.301428


#Adjacency/Sparse Matrix Approach

In [50]:
def dist(lat1, lon1, lat2, lon2):
    R = 6371   # Radius of Earth

    # Convert degrees into radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Calculate the linear distance between two points
    distance = dlat ** 2 + dlon ** 2
    distance = math.sqrt(distance)

    # Calculate spherical distance
    a = math.asin(distance/(2*R))
    c = R * 2 * math.asin(math.sqrt(a))

    return c

In [51]:
num_cities = len(df)
k = 6

# Fit the model on the geographical coordinates
X = df[['latitude', 'longitude']]
nn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(X)

# Find the 5 nearest neighbors
distances, indices = nn.kneighbors(X)

rows, cols, data = [], [], []
seen_edges = set()

for i in range(num_cities):
    for j in range(1, k):
        n_index = indices[i, j]

        #check if updated before
        edge = frozenset((i, n_index))
        if edge in seen_edges:
            continue
        seen_edges.add(edge)

        #If not calc distance
        n_distance = distances[i, j]

        #Add data in both ways to make it undirected
        rows.append(i); cols.append(n_index); data.append(n_distance)
        rows.append(n_index); cols.append(i); data.append(n_distance)

sparse_matrix = csr_matrix((data, (rows, cols)), shape=(num_cities, num_cities))

print("Shape of the sparse adjacency matrix:", sparse_matrix.shape)

Shape of the sparse adjacency matrix: (1919, 1919)


In [52]:
print("\nFirst 10 rows and their non-zero entries (neighbors and distances):")

for i in range(min(10, num_cities)):
    row_data = sparse_matrix.getrow(i)
    print(f"Row {i}: {list(zip(row_data.indices, row_data.data))}")


First 10 rows and their non-zero entries (neighbors and distances):
Row 0: [(np.int32(19), np.float64(0.10280801203432574)), (np.int32(20), np.float64(0.07630243464241533)), (np.int32(32), np.float64(0.09008373774760883)), (np.int32(33), np.float64(0.10312854912632105)), (np.int32(237), np.float64(0.025839241107277235)), (np.int32(256), np.float64(0.019672846708092478))]
Row 1: [(np.int32(3), np.float64(0.0910322710384622)), (np.int32(15), np.float64(0.09149139670707118)), (np.int32(24), np.float64(0.07909985753767391)), (np.int32(34), np.float64(0.07288142004886898)), (np.int32(857), np.float64(0.07483869999999992))]
Row 2: [(np.int32(6), np.float64(0.10433288545808693)), (np.int32(13), np.float64(0.18960060676363566)), (np.int32(14), np.float64(0.16315676726547387)), (np.int32(35), np.float64(0.14410087188900797)), (np.int32(736), np.float64(0.15465547323990886)), (np.int32(1287), np.float64(0.14722019989906115))]
Row 3: [(np.int32(1), np.float64(0.0910322710384622)), (np.int32(5), 

# **Shortest Path Using Dijkstra's Algorithm.**

In [53]:
import heapq
import time

In [54]:
def dijkstra(graph, start_node):
    num_nodes = graph.shape[0]
    distances = [float('inf')] * num_nodes
    distances[start_node] = 0
    priority_queue = [(0, start_node)]
    previous_nodes = [None] * num_nodes

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in zip(graph.indices[graph.indptr[current_node]:graph.indptr[current_node+1]],
                                     graph.data[graph.indptr[current_node]:graph.indptr[current_node+1]]):
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, previous_nodes

In [55]:
def get_shortest_path(previous_nodes, start_node, end_node):
    path = []
    current_node = end_node
    while current_node is not None:
        path.append(current_node)
        current_node = previous_nodes[current_node]
    path.reverse()
    if path and path[0] != start_node:
        return None  # No path found
    return path

# Create a mapping from city name (lowercase) to city index
city_name_to_index = {name.lower(): index for index, name in zip(df['city_id'], df['name_en'])}

# Example Usage (replace start and end nodes with desired city names)
start_city_name = "Akkaraipattu"  # Example: first city in the DataFrame
end_city_name = "Kekirihena" # Example: eleventh city in the DataFrame

# Convert city names to indices
start_node = city_name_to_index.get(start_city_name.lower())
end_node = city_name_to_index.get(end_city_name.lower())

if start_node is None:
    print(f"Start city '{start_city_name}' not found in the dataset.")
elif end_node is None:
    print(f"End city '{end_city_name}' not found in the dataset.")
else:
    start_time = time.time()
    distances, previous_nodes = dijkstra(sparse_matrix, start_node)
    end_time = time.time()

    shortest_distance = distances[end_node]
    shortest_path_indices = get_shortest_path(previous_nodes, start_node, end_node)

    print(f"Execution time for Dijkstra's algorithm: {end_time - start_time:.6f} seconds")

    if shortest_distance != float('inf'):
        print(f"\nShortest distance from {start_city_name} to {end_city_name}: {shortest_distance:.6f}")
        if shortest_path_indices:
            shortest_path_names = [df.loc[i, 'name_en'] for i in shortest_path_indices]
            print(f"Shortest path (city indices): {shortest_path_indices}")
            print(f"Shortest path (city names): {shortest_path_names}")
        else:
            print(f"No path found from {start_city_name} to {end_city_name}")
    else:
        print(f"\nNo path found from {start_city_name} to {end_city_name}")

Execution time for Dijkstra's algorithm: 0.008002 seconds

Shortest distance from Akkaraipattu to Kekirihena: 0.647454
Shortest path (city indices): [0, np.int32(256), np.int32(15), np.int32(857), np.int32(24), np.int32(17), np.int32(1300), 10]
Shortest path (city names): ['Akkaraipattu', 'Navatkadu', 'Irakkamam', 'Hingurana', 'Paragahakele', 'Namaloya', 'Nelliyadda', 'Kekirihena']


### **Taking user input and displaying the distance in kilometers.**

In [56]:
# Get user input for start and end cities
start_city_name_input = input("Enter the starting city name: ")
end_city_name_input = input("Enter the ending city name: ")

# Convert city names to indices (case-insensitive)
start_node_input = city_name_to_index.get(start_city_name_input.lower())
end_node_input = city_name_to_index.get(end_city_name_input.lower())

if start_node_input is None:
    print(f"Start city '{start_city_name_input}' not found in the dataset.")
elif end_node_input is None:
    print(f"End city '{end_city_name_input}' not found in the dataset.")
else:
    start_time = time.time()
    # Use Haversine distance with NearestNeighbors
    nn_haversine = NearestNeighbors(n_neighbors=k, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
    distances_haversine, indices_haversine = nn_haversine.kneighbors(np.radians(X))

    # Rebuild the sparse matrix with Haversine distances (converted to kilometers)
    rows_haversine, cols_haversine, data_haversine = [], [], []
    seen_edges = set()

    for i in range(num_cities):
        for j in range(1, k):
            neighbor_index_haversine = indices_haversine[i, j]

            #check if updated before
            edge = frozenset((i, neighbor_index_haversine))
            if edge in seen_edges:
                continue
            seen_edges.add(edge)

            # Convert haversine distance to kilometers (multiply by Earth's radius in km)
            neighbor_distance_haversine = distances_haversine[i, j] * 6371

            #Add data in both ways to make it undirected
            rows_haversine.append(i); cols_haversine.append(neighbor_index_haversine); data_haversine.append(neighbor_distance_haversine)
            rows_haversine.append(neighbor_index_haversine); cols_haversine.append(i); data_haversine.append(neighbor_distance_haversine)

    sparse_matrix_haversine = csr_matrix((data_haversine, (rows_haversine, cols_haversine)), shape=(num_cities, num_cities))

    distances_input, previous_nodes_input = dijkstra(sparse_matrix_haversine, start_node_input)
    end_time = time.time()

    shortest_distance_input = distances_input[end_node_input]
    shortest_path_indices_input = get_shortest_path(previous_nodes_input, start_node_input, end_node_input)

    print(f"\nExecution time for Dijkstra's algorithm (with Haversine): {end_time - start_time:.6f} seconds")

    if shortest_distance_input != float('inf'):
        print(f"\nShortest distance from {start_city_name_input} to {end_city_name_input}: {shortest_distance_input:.6f} km")
        if shortest_path_indices_input:
            shortest_path_names_input = [df.loc[i, 'name_en'] for i in shortest_path_indices_input]
           # print(f"Shortest path (city indices): {shortest_path_indices_input}")
            print(f"Shortest path (city names): {shortest_path_names_input}")
        else:
            print(f"No path found from {start_city_name_input} to {end_city_name_input}")
    else:
        print(f"\nNo path found from {start_city_name_input} to {end_city_name_input}")


Execution time for Dijkstra's algorithm (with Haversine): 0.060752 seconds

Shortest distance from Madulsima to Akkaraipattu: 85.751943 km
Shortest path (city names): ['Madulsima', 'Arawakumbura', 'Ayiwela', 'Pitakumbura', 'Nilgala', 'Inginiyagala', 'Pahalalanda', 'Hingurana', 'Irakkamam', 'Navatkadu', 'Akkaraipattu']
